In [1]:
import importlib
solver = getattr(importlib.import_module(f"trainer.GAN"), "Solver",)

In [1]:
import torch
from factory.AgainVC import AgainVC

In [2]:
build_config = {'model_name': 'again', 'model': {'params': {'encoder_params': {'c_in': 80, 'c_h': 256, 'c_out': 4, 'n_conv_blocks': 6, 'subsample': [1, 1, 1, 1, 1, 1]}, 'decoder_params': {'c_in': 4, 'c_h': 256, 'c_out': 80, 'n_conv_blocks': 6, 'upsample': [1, 1, 1, 1, 1, 1]}, 'activation_params': {'act': 'sigmoid', 'params': {'alpha': 0.1}}}}, 'optimizer': {'params': {'lr': 0.0005, 'betas': [0.9, 0.999], 'amsgrad': True, 'weight_decay': 0.0001}, 'grad_norm': 3}}

In [7]:
VC = AgainVC(**build_config['model']['params']).to("cuda:0")
VC.load_state_dict(torch.load("model/againvc.pt",map_location="cuda:0"))

<All keys matched successfully>

In [ ]:
agi

In [28]:
d1 = {'a':[1],'b':[3],'c':[1]}
d2 = {'a':[0],'b':[1],'c':[11]}
for k,item in enumerate(d1.items()):
    emb1= item
    key,item = emb1
    #print(emb2)
    print(key)
    print(item)
    print(d2[key])
    break

a
[1]
[0]


In [21]:
import torch
import numpy as np
def get_mel(mels, seq_len, device):
    # input -> (len,80)
    mel, tmp_m = [], []
    pad_size, last_len = 0, 0
    for i in range(0, mels.shape[0], seq_len):
       
        if i + seq_len <= mels.shape[0]:
            print(i)
            mel.append(
                torch.from_numpy(mels[i : i + seq_len, :]).unsqueeze(0).to(device)
            )
            last_len = i + seq_len

    if (mels.shape[0] - last_len) > 30:
        pad_size = int((last_len + seq_len) - mels.shape[0])
        npad = [(0, 0)] * 2
        npad[0] = (0, pad_size)
        tmp_m = np.pad(
            mels[last_len:, :], pad_width=npad, mode="constant", constant_values=0
        )
        mel.append(torch.from_numpy(tmp_m).unsqueeze(0).to(device))

    return mel, pad_size

In [22]:
mels = np.ones((352,80))
mel, pad_size = get_mel(mels, 128, "cpu")
for m in mel:
    print(m.shape)

0
128
torch.Size([1, 128, 80])
torch.Size([1, 128, 80])
torch.Size([1, 128, 80])


In [3]:
import torch 
codes = [torch.rand(1,64)]*8
tmp = []
for code in codes:
    tmp.append(code.unsqueeze(1).expand(-1, int(128 / len(codes)), -1))
code_exp = torch.cat(tmp, dim=1)

In [4]:
code_exp.shape

torch.Size([1, 128, 64])

In [40]:
import random
import numpy as np
from thirdparty.DeepSpeaker.util.audio import read_mfcc,Audio
from thirdparty.DeepSpeaker.util.batcher import sample_from_mfcc
from thirdparty.DeepSpeaker.util.constants import SAMPLE_RATE, NUM_FRAMES
from thirdparty.DeepSpeaker.util.conv_models import DeepSpeakerModel
from thirdparty.DeepSpeaker.util.test import batch_cosine_similarity

In [13]:
waves = Audio.read(f'{ROOT}/samples/p225_014_mic1.flac' ,SAMPLE_RATE)
waves.shape

(87109,)

In [2]:
np.random.seed(123)
random.seed(123)

In [3]:
ROOT = 'thirdparty/DeepSpeaker'

In [4]:
model = DeepSpeakerModel()

In [5]:
model.m.load_weights(f'{ROOT}/model/ResCNN_triplet_training_checkpoint_265.h5', by_name=True)

In [6]:
mfcc_001 = sample_from_mfcc(read_mfcc(f'{ROOT}/samples/PhilippeRemy/PhilippeRemy_001.wav', SAMPLE_RATE), NUM_FRAMES)
mfcc_002 = sample_from_mfcc(read_mfcc(f'{ROOT}/samples/PhilippeRemy/PhilippeRemy_002.wav', SAMPLE_RATE), NUM_FRAMES)
mfcc_003 = sample_from_mfcc(read_mfcc(f'{ROOT}/samples/1255-90413-0001.flac', SAMPLE_RATE), NUM_FRAMES)
mfcc_004 = sample_from_mfcc(read_mfcc(f'{ROOT}/samples/p225_014_mic1.flac', SAMPLE_RATE), NUM_FRAMES)

In [7]:
predict_001 = model.m.predict(np.expand_dims(mfcc_001, axis=0))
predict_002 = model.m.predict(np.expand_dims(mfcc_002, axis=0))
predict_003 = model.m.predict(np.expand_dims(mfcc_003, axis=0))
predict_004 = model.m.predict(np.expand_dims(mfcc_004, axis=0))

In [8]:
print('SAME SPEAKER', batch_cosine_similarity(predict_001, predict_002))
print('DIFF SPEAKER', batch_cosine_similarity(predict_001, predict_003)) 
print('DIFF SPEAKER', batch_cosine_similarity(predict_001, predict_004)) 

SAME SPEAKER [0.8112024]
DIFF SPEAKER [0.02534031]
DIFF SPEAKER [0.18882789]


In [9]:
print('DIFF SPEAKER', batch_cosine_similarity(predict_003, predict_004)) 

DIFF SPEAKER [0.09595115]
